<a href="https://colab.research.google.com/github/guiabbehusen/Projeto-ML-1/blob/main/Models/redes_neurais_recorrentes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
def generate_time_series(batch_size, n_steps):
  freq1, freq2, offset1, offset2 = np.random.rand(4, batch_size, 1)
  time = np.linspace(0, 1, n_steps)
  series = 0.5 * np.sin((time - offset1)* (freq1 * 10 + 10))
  series += 0.2 * np.sin((time - offset1)* (freq1 * 20 + 20))
  series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5)

  return series[..., np.newaxis].astype(np.float32)



In [2]:
import numpy as np
n_steps = 50
series = generate_time_series(10000, n_steps + 1)
X_train, y_train = series[:7000, :n_steps], series[:7000, -1]
X_valid, y_valid = series[7000:9000, :n_steps], series[7000:9000, -1]
X_test, y_test = series[9000:, :n_steps], series[9000:, -1]


In [3]:
y_pred = X_valid[:, -1]

In [4]:
from tensorflow import keras

In [5]:
import tensorflow as tf

In [6]:
mse = tf.keras.metrics.MeanSquaredError()

In [7]:
mse.update_state(y_valid, y_pred)
mse_result = mse.result().numpy()
print(mse_result)

0.020729387


In [8]:
model = keras.models.Sequential([
    keras.layers.SimpleRNN(1, input_shape=[None, 1])
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [9]:
model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20),
    keras.layers.Dense(1)
])

In [10]:
series = generate_time_series(1, n_steps + 10)
X_new, y_new = series[:, :n_steps], series[:, n_steps:]
X = X_new
for step_ahead in range(10):
  y_pred_one = model.predict(X[:, step_ahead:])[:, np.newaxis, :]
  X = np.concatenate([X, y_pred_one], axis=1)
y_pred = X[:, n_steps:]

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step


In [11]:
print(y_pred)

[[[-0.3143639 ]
  [ 0.10389515]
  [ 0.27432817]
  [ 0.7276174 ]
  [-0.13894758]
  [-0.53863955]
  [-0.1367453 ]
  [ 0.7423949 ]
  [ 0.46577245]
  [-0.43437076]]]


In [12]:
print(y_new)

[[[ 0.603273  ]
  [ 0.500939  ]
  [ 0.4488049 ]
  [ 0.39207765]
  [ 0.27742526]
  [ 0.1972701 ]
  [ 0.12089143]
  [ 0.05746683]
  [ 0.06635519]
  [-0.00246681]]]


In [13]:
from sklearn.metrics import mean_squared_error

In [14]:
y_new.shape

(1, 10, 1)

In [15]:
y_pred.shape

(1, 10, 1)

In [16]:
mse = mean_squared_error(y_new.reshape(-1), y_pred.reshape(-1))
print(mse)

0.2739238142967224


In [17]:
series = generate_time_series(10000, n_steps+10)
X_train, Y_train = series[:7000, :n_steps], series[7000:, -10:, 0]
X_valid, Y_valid = series[7000:9000, :n_steps], series[7000:9000, -10:, 0]
X_test, Y_test = series[9000:, :n_steps], series[9000:, -10:, 0]

In [18]:
model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20),
    keras.layers.Dense(10)
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [19]:
Y_pred = model.predict(X_new)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 872ms/step


In [20]:
mse = mean_squared_error(y_new.reshape(-1), Y_pred.reshape(-1))

In [21]:
mse

0.19732244312763214

In [22]:
Y = np.empty((10000, n_steps, 10))
for step_ahead in range(1, 10 +1):
  Y[:, :, step_ahead - 1] = series[:, step_ahead:step_ahead + n_steps, 0]
Y_train = Y[7000:]
Y_valid = Y[7000:9000]
Y_test = Y[9000:]

In [23]:
model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [24]:
def last_step(Y_true, Y_pred):
  return keras.metrics.mean_squared_error(Y_true[:, -1], Y_pred[:, -1])

In [25]:
optimizer = keras.optimizers.Adam(learning_rate=0.01)

In [26]:
model.compile(loss="mse", optimizer=optimizer, metrics=[last_step])


In [27]:
model = keras.models.Sequential([
    keras.layers.LSTM(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.LSTM(20, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

In [29]:
model2 = keras.models.Sequential()

In [30]:
model2.add(keras.layers.InputLayer(input_shape=[None, 1]))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


In [31]:
for rate in (1, 2, 4, 8) * 2:
  model.add(keras.layers.Conv1D(filters=20, kernel_size=2, padding="causal", activation="relu", dilation_rate=rate))

In [32]:
model.add(keras.layers.Conv1D(filters=10, kernel_size=1))

In [33]:
model.compile(loss="mse", optimizer="adam", metrics=[last_step])

ValueError: Data cardinality is ambiguous. Make sure all arrays contain the same number of samples.'x' sizes: 7000
'y' sizes: 3000
